In [10]:
import sys, os
sys.path.append(os.path.abspath("../"))   # make src importable

import pandas as pd
import numpy as np

from src.tdq.tdq_cleaning import (
    remove_duplicates, convert_to_datetime, fix_negative_values,
    fix_invalid_categories, impute_missing, clip_outliers
)

# Load raw data
visitor_df = pd.read_csv("../data/visitor_events.csv")
applications_df = pd.read_csv("../data/applications.csv")
accounts_df = pd.read_csv("../data/accounts.csv")
transactions_df = pd.read_csv("../data/transactions.csv")
marketing_df = pd.read_csv("../data/marketing_source.csv")


In [2]:
os.makedirs("../data/cleaned/", exist_ok=True)


In [3]:
print("Cleaning visitor_events...")

# Convert timestamp
visitor_df = convert_to_datetime(visitor_df, "event_timestamp")

# Remove missing timestamps
visitor_df = visitor_df[visitor_df["event_timestamp"].notnull()]

# Remove duplicates
visitor_df = remove_duplicates(visitor_df)

# Fix invalid marketing source entries
valid_sources = ["email", "ads", "referral", "direct"]
visitor_df = fix_invalid_categories(visitor_df, "marketing_source", valid_sources)

visitor_df.to_csv("../data/cleaned/visitor_events.csv", index=False)

print("✓ visitor_events cleaned")


Cleaning visitor_events...
✓ visitor_events cleaned


In [4]:
print("Cleaning applications...")

# Convert application_date
applications_df = convert_to_datetime(applications_df, "application_date")

# Remove invalid visitor_id
applications_df = applications_df[applications_df["visitor_id"] != 999999]

# Impute credit score
applications_df = impute_missing(applications_df, "credit_score", method="median")

# Clip unrealistic loan amount spikes
applications_df = clip_outliers(applications_df, "loan_amount")

# Remove duplicates
applications_df = remove_duplicates(applications_df)

applications_df.to_csv("../data/cleaned/applications.csv", index=False)

print("✓ applications cleaned")


Cleaning applications...
✓ applications cleaned


/Users/gritv/Desktop/nov 4/src/tdq/tdq_cleaning.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].median(), inplace=True)


In [5]:
print("Cleaning accounts...")

# Convert account_open_date
accounts_df = convert_to_datetime(accounts_df, "account_open_date")

# Fix negative initial deposits
accounts_df = fix_negative_values(accounts_df, "initial_deposit")

# Remove invalid application_ids
valid_app_ids = set(applications_df["application_id"])
accounts_df = accounts_df[accounts_df["application_id"].isin(valid_app_ids)]

# Remove duplicates
accounts_df = remove_duplicates(accounts_df)

accounts_df.to_csv("../data/cleaned/accounts.csv", index=False)

print("✓ accounts cleaned")


Cleaning accounts...
✓ accounts cleaned


In [6]:
print("Cleaning transactions...")

# Convert timestamp
transactions_df = convert_to_datetime(transactions_df, "transaction_timestamp")

# Remove missing timestamps
transactions_df = transactions_df[transactions_df["transaction_timestamp"].notnull()]

# Clip amount spikes
transactions_df = clip_outliers(transactions_df, "amount")

# Remove duplicates
transactions_df = remove_duplicates(transactions_df)

transactions_df.to_csv("../data/cleaned/transactions.csv", index=False)

print("✓ transactions cleaned")


Cleaning transactions...
✓ transactions cleaned


In [7]:
print("Cleaning marketing_source...")

# Fix invalid channel cost (negative values)
marketing_df = fix_negative_values(marketing_df, "channel_cost")

# Impute missing demographics
marketing_df = impute_missing(marketing_df, "target_demographic", method="mode")

marketing_df.to_csv("../data/cleaned/marketing_source.csv", index=False)

print("✓ marketing_source cleaned")


Cleaning marketing_source...
✓ marketing_source cleaned


/Users/gritv/Desktop/nov 4/src/tdq/tdq_cleaning.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace=True)


In [8]:
print("CLEANING COMPLETE!")

for f in os.listdir("../data/cleaned/"):
    print(f, "saved.")


CLEANING COMPLETE!
applications.csv saved.
marketing_source.csv saved.
accounts.csv saved.
visitor_events.csv saved.
transactions.csv saved.


In [14]:
import pandas as pd

# Load RAW data again
raw_visitor = pd.read_csv("../data/visitor_events.csv")
raw_applications = pd.read_csv("../data/applications.csv")
raw_accounts = pd.read_csv("../data/accounts.csv")
raw_transactions = pd.read_csv("../data/transactions.csv")
raw_marketing = pd.read_csv("../data/marketing_source.csv")

raw_data = {
    "visitor_events": raw_visitor,
    "applications": raw_applications,
    "accounts": raw_accounts,
    "transactions": raw_transactions,
    "marketing_source": raw_marketing
}

# Load CLEANED data
clean_visitor = pd.read_csv("../data/cleaned/visitor_events.csv")
clean_app = pd.read_csv("../data/cleaned/applications.csv")
clean_acc = pd.read_csv("../data/cleaned/accounts.csv")
clean_tx = pd.read_csv("../data/cleaned/transactions.csv")
clean_marketing = pd.read_csv("../data/cleaned/marketing_source.csv")

cleaned_data = {
    "visitor_events": clean_visitor,
    "applications": clean_app,
    "accounts": clean_acc,
    "transactions": clean_tx,
    "marketing_source": clean_marketing
}

# Before-after summary
summary_rows = []

for name in raw_data.keys():
    raw_df = raw_data[name]
    clean_df = cleaned_data[name]

    summary_rows.append({
        "table": name,
        "raw_rows": len(raw_df),
        "clean_rows": len(clean_df),
        "rows_removed": len(raw_df) - len(clean_df),
        "raw_nulls": raw_df.isnull().sum().sum(),
        "clean_nulls": clean_df.isnull().sum().sum(),
        "nulls_fixed": raw_df.isnull().sum().sum() - clean_df.isnull().sum().sum(),
        "raw_duplicates": raw_df.duplicated().sum(),
        "clean_duplicates": clean_df.duplicated().sum(),
        "duplicates_removed": raw_df.duplicated().sum() - clean_df.duplicated().sum()
    })

before_after_df = pd.DataFrame(summary_rows)
before_after_df


,table,raw_rows,clean_rows,rows_removed,raw_nulls,clean_nulls,nulls_fixed,raw_duplicates,clean_duplicates,duplicates_removed
0,visitor_events,150050,148999,1051,1001,201,800,50,0,50
1,applications,15050,15000,50,481,0,481,50,0,50
2,accounts,6030,5980,50,0,106,-106,30,0,30
3,transactions,300100,298006,2094,1997,0,1997,100,0,100
4,marketing_source,6,6,0,1,1,0,0,0,0
